In [135]:
# !pip install imblearn

In [136]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [137]:
df = pd.read_csv('loan_data.csv')
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
1,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
2,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
3,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
4,LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y


In [138]:
#Removing useless column
df.drop(['Loan_ID','Gender'],axis=1,inplace=True)
# df.drop(['Loan_ID'],axis=1,inplace=True)


In [139]:
df.isnull().sum()

Married               0
Dependents            8
Education             0
Self_Employed        21
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term     11
Credit_History       30
Property_Area         0
Loan_Status           0
dtype: int64

In [140]:
df.shape

(381, 11)

In [141]:
df = df[~(df.isnull().any(axis=1))].reset_index(drop=True)

In [142]:
df.shape

(313, 11)

In [143]:
df.isnull().any()

Married              False
Dependents           False
Education            False
Self_Employed        False
ApplicantIncome      False
CoapplicantIncome    False
LoanAmount           False
Loan_Amount_Term     False
Credit_History       False
Property_Area        False
Loan_Status          False
dtype: bool

In [144]:
df.head()

,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
1,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
2,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
3,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
4,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y


In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 313 entries, 0 to 312
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Married            313 non-null    object 
 1   Dependents         313 non-null    object 
 2   Education          313 non-null    object 
 3   Self_Employed      313 non-null    object 
 4   ApplicantIncome    313 non-null    int64  
 5   CoapplicantIncome  313 non-null    float64
 6   LoanAmount         313 non-null    float64
 7   Loan_Amount_Term   313 non-null    float64
 8   Credit_History     313 non-null    float64
 9   Property_Area      313 non-null    object 
 10  Loan_Status        313 non-null    object 
dtypes: float64(4), int64(1), object(6)
memory usage: 27.0+ KB


In [146]:
df['Dependents'].value_counts()
df['Dependents'] = df['Dependents'].str.split('+').str[0]
df['Dependents'] = df['Dependents'].astype(int)

In [147]:
cat_coln = [i for i in df.columns if df[i].dtype=='O']
num_coln = [i for i in df.columns if df[i].dtype!='O']

In [148]:
df[cat_coln].describe()

,Married,Education,Self_Employed,Property_Area,Loan_Status
count,313,313,313,313,313
unique,2,2,2,3,2
top,Yes,Graduate,No,Semiurban,Y
freq,188,233,285,126,220


In [149]:
df['Loan_Status'] = df['Loan_Status'].replace({'Y': 1, 'N': 0})

C:\Users\khema\AppData\Local\Temp\ipykernel_18040\3157402979.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Loan_Status'] = df['Loan_Status'].replace({'Y': 1, 'N': 0})


In [150]:
df['Loan_Status'].value_counts()

Loan_Status
1    220
0     93
Name: count, dtype: int64

In [151]:
# sns.pairplot(df,hue='Loan_Status')

In [152]:
X = df.drop(['Loan_Status'],axis=1)
y = df['Loan_Status']

In [153]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [154]:
cat_coln.remove('Property_Area')
cat_coln.remove('Loan_Status')
ohe_coln = ['Property_Area'] 

In [155]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
LE = LabelEncoder()
OHE = OneHotEncoder()
SS = StandardScaler()
for i in cat_coln:
    X_train[i] = LE.fit_transform(X_train[i])
    X_test[i] = LE.transform(X_test[i])
process = ColumnTransformer(
    [
        ("OHE",OHE,ohe_coln),
        ('SS',SS,num_coln)
    ])

In [156]:
X_train = process.fit_transform(X_train)
X_test = process.transform(X_test)

In [157]:
!pip install imblearn
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_train,y_train = oversample.fit_resample(X_train,y_train)

In [158]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
params = {
    'n_estimators': [100,300],
    'criterion': ['gini','entropy','log_loss'],
    'max_depth': [5,10,15,None],
    'min_samples_split': [1,2,3,4],
    'class_weight': ['balanced','balanced_subsample']
}
RFC = RandomForestClassifier(random_state=42)
grid = GridSearchCV(estimator=RFC,param_grid=params,cv=5,n_jobs=-1)
grid.fit(X_train,y_train)
y_pred = grid.predict(X_test)
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(y_test,y_pred),'\n',classification_report(y_test,y_pred),)

c:\Users\khema\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
240 fits failed out of a total of 960.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
240 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\khema\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\khema\anaconda3\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Users\khema\anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "c:\Users\khema\anaconda3\Lib\site-packag

[[13 11]
 [ 4 66]] 
               precision    recall  f1-score   support

           0       0.76      0.54      0.63        24
           1       0.86      0.94      0.90        70

    accuracy                           0.84        94
   macro avg       0.81      0.74      0.77        94
weighted avg       0.83      0.84      0.83        94



In [159]:
grid.best_params_

{'class_weight': 'balanced',
 'criterion': 'gini',
 'max_depth': None,
 'min_samples_split': 4,
 'n_estimators': 100}